In [1]:
%load_ext autoreload
%autoreload 2
import ase
from ase.visualize import view
from ase.build import molecule
from ase.io import Trajectory, write, read
from ase.units import mol # Avogadro Constant
# calculator
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator
# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
import flare
from flare.ase.nosehoover import NoseHoover
from ase.units import kB, fs, Pascal
import numpy as np
import os, shutil

estimator = Estimator()
calculator = ASECalculator(estimator)

def vis(atoms):
    v = view(atoms, viewer='ngl')
    v.view.add_representation("ball+stick")
    display(v)

print(f"pfp_api_client: {pfp_api_client.__version__}")
print(f"Estimator model version: {estimator.model_version}")
print("available versions: ", estimator.available_models)

pfp_api_client: 1.4.0
Estimator model version: v3.0.0
available versions:  ['latest', 'v0.0.0', 'v1.0.0', 'v1.1.0', 'v2.0.0', 'v3.0.0']


In [4]:
## variables for shell scripts ##

# for I/O
INIT_dataf="YSZ_setup01_3x3x3_0.08_NVT_E0_final.traj"
INIT_read_index=":"

OUT_traj="YSZ_setup01_3x3x3_0.08_NVT_Ex0.05.traj"
OUT_log="YSZ_setup01_3x3x3_0.08_NVT_Ex0.05.log"

OUT_traj_itv=100
OUT_log_itv=20

OUT_xyz="YSZ_setup01_3x3x3_0.08_NVT_Ex0.05_charge.xyz"

# condition

INIT_steps=100

INIT_T=800
INIT_P=100000*Pascal
INIT_dt=2*fs

dt = INIT_dt

E_FIELD=0.05

## === parameters for Langevin ====

CELL_a=15.7372
INIT_fric=0.01

## =========================== ##

data_path = INIT_dataf
initial_structure = read(data_path)

data = INIT_dataf

In [6]:
pos = initial_structure

In [7]:
import numpy as np
from langevin_ExternalField import Langevin

T = INIT_T
pos.set_calculator(calculator)

MaxwellBoltzmannDistribution(pos, temperature_K = T)
Stationary(pos)
pos.get_temperature()

efield = np.array([E_FIELD,0,0])
#efield = np.array([0,0,0])
dyn = Langevin(atoms=pos, timestep=dt, 
               temperature_K = T, efield=efield, friction=INIT_fric, trajectory = OUT_traj, loginterval=OUT_traj_itv)

Header= "step etotal T e_pot stress"
print("NVT Simulation")
print(Header)

weight = np.sum(pos.get_masses())*1.673e-27

file = open(OUT_log, "w")
file.write(Header + "\n")
def print_dyn():
    line = f"{dyn.get_number_of_steps(): >3} {pos.get_total_energy():.6f} {pos.get_temperature():.6f} {pos.get_potential_energy():.6f} {-pos.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f}"
    file.write(line+"\n")
    print(line, end = "\r")
    
    
dyn.attach(print_dyn, interval=20)

steps = INIT_steps
dyn.run(steps)
    
file.close()

NVT Simulation
step etotal T e_pot stress


In [8]:
a = read(OUT_traj, index=':')

index = 0
index = 0

def savexyz(traj, filename='pos.xyz'):
    index = 0
    for atoms in traj:
        atoms.wrap()
        atoms.set_calculator(calculator)
        aa=atoms.get_charges()
        #aa = aa.reshape([-1])
        #print(aa)
        #atoms.set_initial_charges( aa )
        write(filename, atoms, append = True)
        index = index + 1

        
savexyz(traj=a, filename=OUT_xyz)
print("generating xyz done!")

generating xyz done!
